### 1. Знакомимся с данными

✍ Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов JOIN.

?
А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

Обратите внимание! Вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке Детализация.
Но прежде, как всегда, разберёмся с датасетом ↓

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

⛏
ЗАГЛЯНЕМ В METABASE!

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим.

Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.



#### Таблица city — это справочник городов. Структура справочника представлена ниже.
НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
- city_id	integer	уникальный идентификатор города, первичный ключ
- city_name	text	название города
- state	text	штат, к которому относится город
- population	integer	население города
- area	numeric	площадь города

#### Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
- cust_id	integer	уникальный идентификатор клиента, первичный ключ
- cust_name	text	название клиента
- annual_revenue	numeric	ежегодная выручка
- cust_type	text	тип пользователя
- address	text	адрес
- zip	integer	почтовый индекс
- phone	text	телефон
- city_id	integer	идентификатор города, внешний ключ к таблице city

#### Следующая таблица — driver — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
- driver_id	integer	уникальный идентификатор водителя, первичный ключ
- first_name	text	имя водителя
- last_name	text	фамилия водителя
- address	text	адрес водителя
- zip_code	integer	почтовый индекс водителя
- phone	text	телефон водителя
- city_id	integer	идентификатор города водителя, внешний ключ к таблице city

#### В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
- truck_id	integer	Уникальный идентификатор грузовика, первичный ключ
- make	text	Производитель грузовика
- model_year	integer	Дата выпуска грузовика

#### Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

НАЗВАНИЕ ПОЛЯ	ТИП ДАННЫХ	ОПИСАНИЕ
- ship_id	integer	уникальный идентификатор доставки, первичный ключ
- cust_id	integer	идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer
- weight	numeric	вес посылки
- truck_id	integer	идентификатор грузовика, на котором отправлена доставка, внешний ключ
к таблице truck
- driver_id	integer	идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver
- city_id	integer	идентификатор города в который совершена доставка, внешний ключ
к таблице city
- ship_date	date	дата доставки

#  2. UNION
### ПРИНЦИП И УСЛОВИЯ РАБОТЫ UNION

✍ Вернёмся к центральному вопросу модуля: как соединить несколько результатов, чтобы получить в выводе один общий?

Чтобы разобраться в этом вопросе, смоделируем ситуацию.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

In [ ]:
# SELECT          book_name object_name, 'книга' object_description 
# FROM          public.books
# UNION ALL
# SELECT          movie_title, 'фильм' 
# FROM          sql.kinopoisk

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:

1. одинаковый тип данных;
2. одинаковое количество столбцов;
3. одинаковый порядок столбцов согласно типу данных.

### ВИДЫ UNION

Оператор присоединения существует в двух вариантах:

- UNION выводит только уникальные записи;
- UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.

### СИНТАКСИС

Запрос строится таким образом:

In [ ]:
# SELECT         n columns
# FROM 
#          table_1
# UNION ALL
# SELECT 
#          n columns
# FROM 
#          table_2
# ...
# UNION ALL
# SELECT 
#          n columns
# FROM 
#          table_n

Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.

### Важно!
Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.

Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.

Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.



In [ ]:
# SELECT
#          c.city_id object_name, 'id города' object_type
# FROM 
#          sql.city c
# UNION ALL
# SELECT
#          d.driver_id other_name, 'id водителя' other_type
# FROM 
#          sql.driver d
# UNION ALL
# SELECT
#          s.ship_id, 'id доставки'
# FROM 
#          sql.shipment s
# UNION ALL
# SELECT
#          c.cust_id, 'id клиента'
# FROM 
#          sql.customer c
# UNION ALL
# SELECT
#          t.truck_id, 'id грузовика'
# FROM 
#          sql.truck t
# ORDER BY 1

В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

In [ ]:
# (SELECT book_name object_name, 'книга' object_descritption 
# FROM public.books
# ORDER BY 1
# LIMIT 1)
# UNION ALL
# (SELECT movie_title, 'фильм' 
# FROM sql.kinopoisk
# ORDER BY 1
# LIMIT 1)

### Задание 2.1
Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

In [ ]:
# SELECT c.city_name "название", 'city' "тип объекта"
# FROM sql.city c
# UNION
# SELECT c.state, 'state'
# FROM sql.city c
# UNION
# SELECT d.first_name, 'driver'
# FROM sql.driver d
# UNION
# SELECT t.make, 'truck'
# FROM sql.truck t
# ORDER BY 1,2


### Задание 2.2

Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

In [ ]:
# SELECT
#     c.city_name AS object_name
# FROM    
#     sql.city c
# UNION ALL

# SELECT
#     c.state AS object_name
# FROM 
#     sql.city c
    

### Задание 2.3

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

In [ ]:
# SELECT DISTINCT
#     c.city_name AS object_name
# FROM    
#     sql.city c
# UNION
# 
# SELECT
#     c.state AS object_name
# FROM 
#     sql.city c
# ORDER BY 
#     object_name ASC;

## 3. UNION и ограничение типов данных
### ПОЧЕМУ ТАК ВАЖЕН ТИП ДАННЫХ?

Как мы уже знаем, UNION может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Допустим, мы хотим вывести список всех id городов и их названий в одном столбце.

⛏
ПОПРОБУЙТЕ В METABASE!

Давайте напишем запрос, который позволит получить нужный нам результат.



- SELECT
- -------c.city_id
- FROM
- -------sql.city c
- UNION ALL
- SELECT 
- -------cc.city_name
- FROM
- -------sql.city cc

Вместо результата вы получите сообщение об ошибке: "ERROR: UNION types integer and text cannot be matched". Дело в том, что мы попытались объединить числовой и строковый типы в одной колонке, а это невозможно.

Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в Postgres составляется запрос по модели column_name::column_type.

Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text.

Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.



Немного подправим запрос, чтобы получить желаемый результат.

⛏
ПОПРОБУЙТЕ В METABASE!

ЗапросДетализация
- SELECT 
- ---------c.city_id::text
- FROM
- ---------sql.city c
- UNION ALL
- SELECT 
- ---------cc.city_name
- FROM
- ---------sql.city cc

### Задание 3.1
Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

- SELECT
- --d.zip_code::text contact,
- --d.first_name first_name,
- --'zip' contact_type
- FROM
- --sql.driver d
- UNION
- SELECT
- --dd.phone contact,
- --dd.first_name first_name,
- --'phone' contact_type
- FROM
- --sql.driver dd
- ORDER BY 1,2




### 4. UNION ALL и промежуточные итоги
ВОЗМОЖНОСТИ UNION

✍ Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.


Попробуем вывести обобщённые данные о населении по всем городам, с детализацией до конкретного города.

⛏
ПОПРОБУЙТЕ В METABASE!


- SELECT
- -------c.city_name,
- -------c.population
- FROM
- -------sql.city c
- UNION ALL
- SELECT
- -------'total',
- -------SUM(c.population)
- FROM
- -------sql.city c
- ORDER BY 2 DESC
Визуально это действие можно представить так:

### Задание 4.1

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipment. Не забывайте о единой типизации. Упорядочите по убыванию столбца date_period.

- SELECT s.ship_date::text date_period, COUNT(*) cnt_shipment
- FROM sql.shipment s 
- GROUP BY 1
- UNION ALL
- SELECT 'total_shipments', COUNT(*)
- FROM sql.shipment s
- ORDER BY 1 desc


### 5. UNION и дополнительные условия

✍ UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

Например, с помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона.

⛏
ПОПРОБУЙТЕ В METABASE!


- SELECT
- ------- d.first_name,
- ------- d.last_name,
- ------- 'телефон заполнен' phone_info
- FROM
- ------- sql.driver d
- WHERE d.phone IS NOT NULL

- UNION

- SELECT
- ------- d.first_name,
- ------- d.last_name,
- ------- 'телефон не заполнен' phone_info
- FROM
- ------- sql.driver d
- WHERE d.phone IS NULL

### Задание 5.1

Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:

если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
если нет — выводим 'доставка не осуществлялась'.
Столбцы к выводу: city_name, state, shipping_status. Отсортируйте в алфавитном порядке по городу, а затем — по штату.

- SELECT 
- ----c.city_name AS city_name,
- ----c.state AS state,
- ----'доставка осуществлялась' AS shipping_status
- FROM 
- ----sql.city c
- ----LEFT JOIN sql.shipment s ON c.city_id=s.city_id
- WHERE s.city_id IS NOT NULL
- UNION
- SELECT 
- ----c.city_name AS city_name,
- ----c.state AS state,
- ----'доставка не осуществлялась' AS shipping_status
- FROM 
- ----sql.city c
- ----LEFT JOIN sql.shipment s ON c.city_id=s.city_id
- WHERE s.city_id IS NULL
- ORDER BY 1, 2


### Задание 5.2
Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:

если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
иначе — вывести количество доставок, увеличенное на пять.
Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

- SELECT
-     c.city_name AS city_name,
-     COUNT(s.ship_id) shippings_fake
- FROM
-     sql.city c
-     JOIN sql.shipment s ON c.city_id=s.city_id
- GROUP BY
-     c.city_name
- HAVING
-      COUNT(s.ship_id) > 10
- UNION
- SELECT
-     c.city_name AS city_name,
-     COUNT(s.ship_id)+5 shippings_fake
- FROM
-     sql.city c
-     JOIN sql.shipment s ON c.city_id=s.city_id
- GROUP BY
-     c.city_name
- HAVING
-      COUNT(s.ship_id) <= 10
- ORDER BY
-     shippings_fake desc,
-     city_name asc


### 6. UNION и ручная генерация     

✍ UNION можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.
Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.


- SELECT 
- -------'a' letter,'1' ordinal_position
- UNION 
- SELECT
- -------'b','2'
- UNION 
- SELECT
- -------'c','3'

Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы unicode и позиции элемента в ней с учётом определённых условий.

### Задание 6.1
Напишите запрос, который выберет наибольшее из значений:

- 1000000;
- 541;
- -500;
- 100.

Столбец с результатом назовите result.

- SELECT  1000000 result
- UNION ALL
- SELECT  541
- UNION ALL
- SELECT  -500
- UNION ALL
- SELECT  100
- ORDER BY 1 DESC 
- LIMIT 1


### Задание 6.2
Помним, что сортировка для числовых и строковых типов данных отличается. Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:

1000000;
541;
-500;
100.
Столбец с ответом назовите mycol

- SELECT 1000000::text AS mycol
- UNION ALL
- SELECT 541::text
- UNION ALL
- SELECT '-500'::text
- UNION ALL
- SELECT 100::text
- ORDER BY mycol DESC
- LIMIT 1;


### Задание 6.3

Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:

"+" ;
"-" ;
"=" ;
"/" .
Столбец с ответом назовите result.

- SELECT '+' AS result
- UNION ALL
- SELECT '-'
- UNION ALL
- SELECT '='
- UNION ALL
- SELECT '/'
- ORDER BY result DESC
- LIMIT 1;

### 7. EXCEPT
ИСКЛЮЧАЕМ ПОВТОРЯЮЩИЕСЯ ДАННЫЕ

Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.

⛏
ПОПРОБУЙТЕ В METABASE!
 
- SELECT c.city_name
- FROM sql.shipment s
- JOIN sql.city c ON s.city_id = c.city_id
- EXCEPT
- SELECT cc.city_name
- FROM sql.driver d 
- JOIN sql.city cc ON d.city_id=cc.city_id
- ORDER BY 1

Все водители проживают в городе Memphis, и мы видим, что он не выводится в результате запроса.

Как вы, должно быть, заметили, для решения этой задачи мы использовали оператор EXCEPT. Принципы его работы мы сейчас разберём ↓



Синтаксические правила для оператора EXCEPT такие же, как и для UNION:

- одинаковый тип данных;
- одинаковое количество столбцов;
- одинаковый порядок столбцов согласно типу данных.
- Синтаксис выглядит следующим образом:

- SELECT n columns
- FROM table_1
- EXCEPT
- SELECT n columns
- FROM  table_2

Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.

Предположим, у нас есть информация о продажах канцелярского магазина за май и июнь.

Какие-то позиции продавались и в том, и в другом месяце, а какие-то — только в одном. Использовав EXCEPT, мы можем оставить только те товары, которые есть в первом запросе (например, за май), но отсутствуют во втором запросе (например, за июнь).

### Задание 7.1
Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip. В поле ниже введите запрос, с помощью которого вы решили эту задачу.

- SELECT zip_code AS zip
- FROM sql.driver
- EXCEPT 
- SELECT zip 
- FROM sql.customer
- ORDER BY 1


##  8. INTERSECT
### ВЫБИРАЕМ ОБЩИЕ ДАННЫЕ

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах?

Предположим, нам надо вывести совпадающие по названию города и штаты.

Запрос

- SELECT          c.city_name object_name
- FROM          sql.city c
- INTERSECT
- SELECT cc.state
- FROM          sql.city cc
- ORDER BY 1

Как видим, с помощью оператора INTERSECT мы вывели названия городов и штатов, которые совпадают: New York, Washington и Wyoming. Присмотримся к нему внимательнее.



Чтобы лучше понять, как работает этот оператор, вновь обратимся к диаграмме Венна: INTERSECT оставляет из результатов первого запроса все строки, которые совпали с результатом выполнения второго запроса.


Синтаксис запроса с оператором INTERSECT выглядит следующим образом:

- SELECT 
- -------n columns
- FROM 
- -------table_1
- INTERSECT
- SELECT 
- -------n columns
- FROM 
- -------table_2

Оператор INTERSECT оставляет только те строки, которые являются общими для двух запросов.

#### Как EXCEPT, так и INTERSECT убирают дубликаты, если они имеются.

### Задание 8.1
Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.

- SELECT c.city_id 
- FROM sql.customer c 
- INTERSECT
- SELECT  cc.city_id
- FROM  sql.shipment cc
- INTERSECT
- SELECT ccc.city_id
- FROM sql.driver ccc


### Задача 8.2
Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.

- SELECT d.zip_code 
- FROM sql.driver d 
- INTERSECT
- SELECT c.zip
- FROM sql.customer c

### 9. Итоги. Закрепление знаний
✍ В этом модуле вы освоили ещё несколько возможностей из функционала SQL.

Теперь вы умеете:

присоединять таблицы «снизу», одну под другой;
исключать или, наоборот, выбирать повторяющиеся записи из двух таблиц;
вручную добавлять записи в таблицу.
Запишем структуру запроса с учётом полученных знаний.

- SELECT--------N columns
- FROM----------table_1
- UNION / UNION ALL / EXCEPT / INTERSECT 
- SELECT--------N columns
- FROM----------table_2

### Задание 9.1
Выведите города с максимальным и минимальным весом единичной доставки. Столбцы к выводу — city_name, weight.

- (
- -----SELECT
- -------c.city_name,
- -------s.weight
- -----FROM sql.shipment s
- -----JOIN sql.city c ON s.city_id=c.city_id
- ---ORDER BY 2 desc
- ---LIMIT 1
- )
- UNION ALL
- (
- ---SELECT
- -------c.city_name,
- -------s.weight
- ---FROM sql.shipment s
- -------JOIN sql.city c ON s.city_id=c.city_id
- ---ORDER BY 2
- ---LIMIT 1
- )


### Задание 9.2 
Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id). Столбец к выводу — mutual_id. Отсортируйте по возрастанию.

- SELECT c.cust_id AS mutual_id
- FROM sql.customer c
- INTERSECT 
- SELECT s.ship_id
- FROM sql.shipment s
- ORDER BY mutual_id ASC;


### Задание 9.3 

Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'. Столбцы к выводу — object_name, object_description. Отсортируйте по названию в алфавитном порядке.

- SELECT cust_name object_name, 'КЛИЕНТ' object_description
- FROM sql.customer
- WHERE cust_type='manufacturer'
- UNION
- SELECT make, 'ГРУЗОВИК'
- FROM sql.truck
- ORDER BY 1
